In [1]:
import re
import pandas as pd
import urllib
from numpy import NaN

In [2]:
with open("dosp_29012016.csv") as file:
    dt = pd.read_csv(file, index_col='ID')
    print(dt)

                                Data Retranca Tipo do Conteúdo  \
ID                                                               
2016/01/29/0    2016-01-29T00:00:00Z  efmaadm       LICITAÇÕES   
2016/01/29/1    2016-01-29T00:00:00Z  efaaadm       LICITAÇÕES   
2016/01/29/2    2016-01-29T00:00:00Z  efmaadm       LICITAÇÕES   
2016/01/29/3    2016-01-29T00:00:00Z  epaaadm       LICITAÇÕES   
2016/01/29/4    2016-01-29T00:00:00Z  epwbadm       LICITAÇÕES   
...                              ...      ...              ...   
2016/01/29/702  2016-01-29T00:00:00Z  dseaadm          EDITAIS   
2016/01/29/703  2016-01-29T00:00:00Z  bpaaadm       SERVIDORES   
2016/01/29/704  2016-01-29T00:00:00Z  bofaadm       SERVIDORES   
2016/01/29/705  2016-01-29T00:00:00Z  abnxidm         DESPACHO   
2016/01/29/706  2016-01-29T00:00:00Z  agidadm         DESPACHO   

                                    Secretaria  \
ID                                               
2016/01/29/0                             

In [3]:
data_dict = {'Título da Publicação': [],
             'Número do Processo': [],
             'Valor (Valor Global) em R$': []
             }
print(data_dict)

{'Título da Publicação': [], 'Número do Processo': [], 'Valor (Valor Global) em R$': []}


In [4]:
counter = 0
# Limpando o dataset, para varias execucoes deste mesmo bloco
for k in data_dict.keys():
    data_dict[k].clear()

limite = 10
print(f"{limite} primeiros valores encontrados:\n")
print(data_dict.keys(), '\n')

for texto in dt['Texto']:
    titulo = re.search(r'(t[íi]tulo|T[ÍI]TULO)\){1,2}[^\(\)\n]+[\s\(]', texto)

    if titulo:
        data_dict['Título da Publicação'].append(re.sub(r'(t[íi]tulo|T[ÍI]TULO)\){1,2}\s*|[\s\(]$', '', titulo.group()))
    else:
        data_dict['Título da Publicação'].append(NaN)

    n_processo = re.search(r'\d{4}[\-\.]\d\.(\d{1,3}[\-\.])+\d|(\d{4}\.?)+/\d+\-\d', texto)

    if n_processo:
        data_dict['Número do Processo'].append(n_processo.group())
    else:
        data_dict['Número do Processo'].append(NaN)

    valor = re.search(r'([Vv]alor|VALOR).*?(R\$ *(\d{1,3}\.?)+,\d{2})', texto)

    if(valor):
        data_dict['Valor (Valor Global) em R$'].append(re.search(r'(\d{1,3}\.?)+,\d{2}', valor.group()).group())
    else:
        data_dict['Valor (Valor Global) em R$'].append(NaN)

    if(counter < 10):
        for k in data_dict.keys():
            print(data_dict[k][counter], end='\t')

        print()
        
    counter += 1

10 primeiros valores encontrados:

dict_keys(['Título da Publicação', 'Número do Processo', 'Valor (Valor Global) em R$']) 

Extrato de Termo de Aditamento	2013-0.009.972-7	14.100,00	
CONTRAÇÃO DE NATUREZA ARTÍSTICA	6025.2016/0000082-0	14.000,00	
Extrato de Termo de Aditamento	2013-0.009.972-7	14.100,00	
PROCESSO: 2015-0.272.291-3	2015-0.272.291-3	30.560,00	
PENALIDADE	2015-0.245.159-6	nan	
2015-0.038.991-5	2015-0.038.991-5	nan	
DISPENSA DE LICITAÇÃO	6025.2016/0000004-8	11.999.014,74	
EXTRATO DE ORDEM DE EXCECUÇÃO DE SERVIÇO	2015-0.336.332-1	nan	
CONTRAÇÃO DE NATUREZA ARTÍSTICA	6025.2016/0000078-1	55.000,00	
CONTRAÇÃO DE NATUREZA ARTÍSTICA	6025.2016/0000086-2	15.000,00	


In [5]:
# Convertendo o resultado pro dataset do pandas
result_df = pd.DataFrame(data_dict)
print(result_df)

                           Título da Publicação   Número do Processo  \
0                Extrato de Termo de Aditamento     2013-0.009.972-7   
1               CONTRAÇÃO DE NATUREZA ARTÍSTICA  6025.2016/0000082-0   
2                Extrato de Termo de Aditamento     2013-0.009.972-7   
3                    PROCESSO: 2015-0.272.291-3     2015-0.272.291-3   
4                                    PENALIDADE     2015-0.245.159-6   
..                                          ...                  ...   
701                                         NaN     2015-0.218.431-8   
702      COORDENAÇÃO DE GESTÃO DE PESSOAS - CGP     2015-0.323.706-7   
703                       LICENÇA MÉDICA - RGPS                  NaN   
704  Processo Eletrônico nº 6074.2016/0000050-9  6074.2016/0000050-9   
705   DIRETORIA REGIONAL DE EDUCAÇÃO - PIRITUBA     2016-0.010.708-3   

    Valor (Valor Global) em R$  
0                    14.100,00  
1                    14.000,00  
2                    14.100,00  
3  

In [6]:
# Salvando
result_df.to_csv('dosp_processado.csv')